In [3]:
from math import pi
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import Range1d, LinearAxis, HoverTool
from bokeh.layouts import column
from datetime import datetime, timedelta
import sys
import backtest_ema2


In [4]:
def recent_gdax(sd_str):
    engine = ms.connect_mysql()
    gdax_df = ms.get_gdax_prices(sd_str)
    return(gdax_df)



def df_resolution(trimmed_df, res_type):
    if res_type == 'day':
        df = trimmed_df.groupby(trimmed_df['time'].dt.date).agg(
    {'open':'first',  'high': max, 'low': min, 'close':'last',
     'volumeto': sum, 'volumefrom': sum,
    }).reset_index()
    elif res_type == 'minute':
        df = trimmed_df
    return(df)



def combine_data_sources():
    raw_data = pd.read_csv('~/twoseven.csv')
    coinbase_data = pd.read_csv('~/coinbase_data.csv')
    raw_min = raw_data.time.min()
    coinbase_data = coinbase_data[coinbase_data['time'] < raw_min]
    combined_df = pd.concat([coinbase_data, raw_data])
    return(combined_df)

In [5]:
def graph_candlestick(full_df, candle_df, fills, results):
    buys = fills[fills['side']=='buy']
    sells = fills[fills['side']=='sell']
    inc = candle_df.close > candle_df.open
    dec = candle_df.open > candle_df.close
    #w = 18*60*60*1000 # candle period in ms
    res_len_secs = (candle_df.time[2] - candle_df.time[1]).seconds
    w = res_len_secs * 1000 # candle period in ms
    hover = HoverTool(tooltips = [
           ("(x,y)", "($x, $y)"),
        ],
        formatters={
            'time': 'datetime', # use 'datetime' formatter for 'date' field
        },
    )
    TOOLS = "crosshair,wheel_zoom,pan,box_zoom,reset,save"
    if results is not None:
        my_title = f"Final Balance: {round(results['final_bal'],2)}, ROI: {round(results['roi'],2)}, Hodl ROI: {round(results['hodl_roi'],2)}"
    else:
        my_title = "BTC Minute Data"
    p = figure(x_axis_type="datetime", tools=[hover,TOOLS], active_scroll="wheel_zoom", plot_width=800, title = my_title,  y_range=(candle_df['close'].min()*.9, candle_df['close'].max() * 1.1), logo=None)
    p.grid.grid_line_alpha=0.3
    band_x = np.append(full_df.time, full_df.time[::-1])
    band_y = np.append(full_df.lower_ema, full_df.upper_ema[::-1])
    p.patch(band_x, band_y, color='purple', fill_alpha=0.03)
    p.segment(candle_df.time, candle_df.high, candle_df.time, candle_df.low, color="black", alpha = 0.5)
    p.vbar(candle_df.time[inc], w, candle_df.open[inc], candle_df.close[inc], fill_color="#D5E1DD", line_color="green", alpha = 0.5)
    p.vbar(candle_df.time[dec], w, candle_df.open[dec], candle_df.close[dec], fill_color="#F2583E", line_color="red", alpha = 0.5)
    #p.multi_line([list(df.time),list(df.time)],[list(df.r30),list(df.r3)], line_color=['blue', 'orange'])
    p.line(full_df.time, full_df.upper_ema, color="purple", alpha = 0.02)
    p.line(full_df.time, full_df.lower_ema, color="purple", alpha = 0.02)
    #p.line(full_df.time, full_df.r3, color="red", alpha = 0.3)
    #if full_df is not None:
    #    p.line(full_df.time, full_df.r30, color="purple", alpha = 0.3)
    #p.extra_y_ranges = {"foo": Range1d(start=0, end=df.n.max())}
    #p.line(df.time, df.n, color="blue", y_range_name="foo", alpha = 0.4)
    #p.add_layout(LinearAxis(y_range_name="foo"), 'left')
    p.circle(buys['time'], buys['price'], color="green", size=8, alpha=0.5)
    p.circle(sells['time'], sells['price'], color="red", size=8, alpha=0.5)
    return(p)

In [6]:
start_time = "2017-01-01"
end_time = "2018-05-06"
raw_data = combine_data_sources()
day_df, trimmed_df = backtest_ema2.prep_data(raw_data, start_time, end_time)

In [7]:
bt_vars = backtest_ema2.BacktestSettings()

upper_factor = 1 + .02
lower_factor = 1 - .02
bt_vars.set_factor_high(upper_factor)
bt_vars.set_factor_low(lower_factor)
bt_vars.set_upper_window(600)
bt_vars.set_lower_window(300)
bt_vars.set_sell_pct_btc(1)
bt_vars.set_buy_amt_usd(250)


bt_vars.set_min_usd(500)
bt_vars.set_min_btc(.001)
bt_vars.set_principle_usd(25000)


#raw_data = pd.read_csv('~/coinbase_data.csv')
#raw_data = recent_gdax(start_time)


In [9]:
raw_data.head()

,close,high,low,mean,open,time,volume,volumefrom,volumeto
0,300.0,300.0,300.0,300.0,300.0,2014-12-01 13:33:00,NaN,3.0,0.01
1,300.0,300.0,300.0,300.0,300.0,2014-12-01 13:34:00,NaN,3.0,0.01
2,300.0,300.0,300.0,300.0,300.0,2014-12-01 13:35:00,NaN,3.0,0.01
3,300.0,300.0,300.0,300.0,300.0,2014-12-01 13:36:00,NaN,3.0,0.01
4,300.0,300.0,300.0,300.0,300.0,2014-12-01 13:37:00,NaN,3.0,0.01


In [14]:
raw_data.to_csv("~/backfire/data/coinbase_2014-12-01_2018-05-06.csv", index=False)

In [14]:
trimmed_df, my_res, my_fills = backtest_ema2.single_backtest(
    trimmed_df, bt_vars)




In [16]:
pd.DataFrame([my_res])[['sd', 'ed', 'factor_high', 'factor_low', 'upper_window', 'lower_window', 'hodl_roi', 'roi']]

,sd,ed,factor_high,factor_low,upper_window,lower_window,hodl_roi,roi
0,2017-01-01,2018-05-06,1.02,0.98,600,300,9.185099,46.208388


In [ ]:
#p = graph_candlestick(trimmed_df, day_df, my_fills, my_res)
#output_notebook()
#show(p)

